Linear regression model that predicts the amount of views a YouTube video will get

In the realm of AI/ML engineering, as valuable it is to be able to curate your own dataset, it is an equally important skill to be able to find (usually publically available, but essentially any that aren't created by you) datasets that provide high-quality, clean data that is large enough in amounts to ensure your model has the data it needs to be able to churn out highly-accurate outputs.

For this YouTube regression model, we are going to use a publically available dataset from a popular source, Kaggle.

In [3]:
# first we need to install the kaggle package
%pip install tensorflow
%pip install kaggle
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
# then we need to set up the Kaggle API credentials
import kaggle
import os
# then you need to go to the actual kaggle website (https://www.kaggle.com/settings, if you're not signed in, you'll have to sign in before accessing that page), create a new token, and then, once the website prompts you to save a json file, save it into the correct directory (usually that would be Users/your_user/.kaggle/kaggle.json).
# once you've saved the kaggle.json file into the correct directory, you can set the environmental variables for the kaggle username (and the key for that respective kaggle account) before using the kaggle library.
os.environ["KAGGLE_USERNAME"] = "your_username"
os.environ["KAGGLE_KEY"] = "your_key"

In [1]:
import kaggle
import os
os.environ["KAGGLE_USERNAME"] = "atinkumarsingh"
os.environ["KAGGLE_KEY"] = "e28a6fc4eb1b5f96e7a1cd388152f7cc"

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
# then we use the Kaggle API to download the dataset
api = KaggleApi()
api.authenticate()

dataset_name = 'rsrishav/youtube-trending-video-dataset'
# Our dataset is 'rsrishav/youtube-trending-video-dataset'
# For some datasets, Kaggle creates a zip file of the dataset, and uploads it to Google Cloud Storage.
# The problem is, Kaggle only does this for SOME datasets, not all datasets.
# The dataset we are using does not have a zip file like this.
# If it DID have a zip file like that, we could use the following code to download it
# api.dataset_download_files('rsrishav/youtube-trending-video-dataset', path='data/', unzip=True)

# Since it does not, our only other option is to download directly from the Kaggle dataset page, or by setting up the Kaggle API, as we did above, and the Kaggle CLI.
# From a brief look at it, setting up the Kaggle CLI seems to be a time-consuming process on a notebook, so I might add that later.
# For now, we will download the dataset directly from the Kaggle dataset page.
# The dataset is available at https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset
# I have downloaded the dataset manually and saved it to /data/GB_youtube_trending.data.csv


If any of the above does not work for you, don't worry. This repository comes with all datasets downloaded with it, so you can skip the Kaggle stage in case its taking too much your time and move on to the rest of this notebook.

In [3]:
# Now we're going to read the dataset (using pandas)
import pandas as pd
df = pd.read_csv('data/GB_youtube_trending_data.csv')

In [4]:
# Here's a quick look at the first 10 rows of our dataset
df.head(10)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,J78aPJ3VyNs,I left youtube for a month and THIS is what ha...,2020-08-11T16:34:06Z,UCYzPXprvl5Y-Sf0g4vX-m6g,jacksepticeye,24,2020-08-12T00:00:00Z,jacksepticeye|funny|funny meme|memes|jacksepti...,2038853,353790,2628,40228,https://i.ytimg.com/vi/J78aPJ3VyNs/default.jpg,False,False,I left youtube for a month and this is what ha...
1,9nidKH8cM38,TAXI CAB SLAYER KILLS 'TO KNOW HOW IT FEELS',2020-08-11T20:00:45Z,UCFMbX7frWZfuWdjAML0babA,Eleanor Neale,27,2020-08-12T00:00:00Z,eleanor|neale|eleanor neale|eleanor neale true...,236830,16423,209,1642,https://i.ytimg.com/vi/9nidKH8cM38/default.jpg,False,False,The first 1000 people to click the link will g...
2,M9Pmf9AB4Mo,Apex Legends | Stories from the Outlands – “Th...,2020-08-11T17:00:10Z,UC0ZV6M2THA81QT9hrVWJG3A,Apex Legends,20,2020-08-12T00:00:00Z,Apex Legends|Apex Legends characters|new Apex ...,2381688,146739,2794,16549,https://i.ytimg.com/vi/M9Pmf9AB4Mo/default.jpg,False,False,"While running her own modding shop, Ramya Pare..."
3,kgUV1MaD_M8,Nines - Clout (Official Video),2020-08-10T18:30:28Z,UCvDkzrj8ZPlBqRd6fIxdhTw,Nines,24,2020-08-12T00:00:00Z,Nines|Trapper of the year|Crop Circle|Nines Tr...,613785,37567,669,2101,https://i.ytimg.com/vi/kgUV1MaD_M8/default.jpg,False,False,Nines - Clout (Official Video)Listen to Clout ...
4,49Z6Mv4_WCA,i don't know what im doing anymore,2020-08-11T20:24:34Z,UCtinbF-Q-fVthA0qrFQTgXQ,CaseyNeistat,22,2020-08-12T00:00:00Z,[None],940036,87113,1860,7052,https://i.ytimg.com/vi/49Z6Mv4_WCA/default.jpg,False,False,ssend love to my sponsor; for a super Limited ...
5,ua4QMFQATco,CGP Grey was WRONG,2020-08-11T17:15:11Z,UC2C_jShtL725hvbm1arSV9w,CGP Grey,27,2020-08-12T00:00:00Z,cgpgrey|education|hello internet,1050143,89192,855,6455,https://i.ytimg.com/vi/ua4QMFQATco/default.jpg,False,False,‣ What Was TEKOI: https://www.youtube.com/watc...
6,x-KbnJ9fvJc,Kya Baat Aa : Karan Aujla (Official Video) Tan...,2020-08-11T09:00:11Z,UCm9SZAl03Rev9sFwloCdz1g,Rehaan Records,10,2020-08-12T00:00:00Z,[None],11308046,655449,33242,405146,https://i.ytimg.com/vi/x-KbnJ9fvJc/default.jpg,False,False,Singer/Lyrics: Karan Aujla Feat Tania Music/ D...
7,3C66w5Z0ixs,I ASKED HER TO BE MY GIRLFRIEND...,2020-08-11T19:20:14Z,UCvtRTOMP2TqYqu51xNrqAzg,Brawadis,22,2020-08-12T00:00:00Z,brawadis|prank|basketball|skits|ghost|funny vi...,1514614,156910,5856,35313,https://i.ytimg.com/vi/3C66w5Z0ixs/default.jpg,False,False,SUBSCRIBE to BRAWADIS ▶ http://bit.ly/Subscrib...
8,ZNfeMbO_AHo,Popek ft. Dr Alban - It's My Life (prod. Clay...,2020-08-12T10:00:09Z,UC8Mh9UmrIaQPEcybdWvQsOg,KrólAlbaniiTV,24,2020-08-12T00:00:00Z,[None],277506,27420,617,1268,https://i.ytimg.com/vi/ZNfeMbO_AHo/default.jpg,False,False,Nowa wersja kultowego utworu z lat 90’.Posłuch...
9,VIUo6yapDbc,Ultimate DIY Home Movie Theater for The LaBran...,2020-08-11T15:10:05Z,UCDVPcEbVLQgLZX0Rt6jo34A,Mr. Kate,26,2020-08-12T00:00:00Z,The LaBrant Family|DIY|Interior Design|Makeove...,1123889,45803,964,2198,https://i.ytimg.com/vi/VIUo6yapDbc/default.jpg,False,False,Transforming The LaBrant Family's empty white ...


In [5]:
# And some stats
df.describe()

,categoryId,view_count,likes,dislikes,comment_count
count,268791.000000,2.687910e+05,2.687910e+05,268791.000000,2.687910e+05
mean,19.046270,2.190558e+06,1.080977e+05,975.519404,7.982660e+03
std,6.380657,6.651493e+06,3.465319e+05,7288.658511,5.953216e+04
min,1.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,17.000000,3.435020e+05,1.109500e+04,0.000000,7.650000e+02
50%,20.000000,7.745050e+05,3.213700e+04,0.000000,1.978000e+03
75%,24.000000,1.844036e+06,8.720100e+04,275.000000,5.071000e+03
max,29.000000,1.406330e+09,1.524651e+07,865075.000000,5.987770e+06


If you remember, what we are going to do is predict the view count. So we're first going to clean the data, before setting up the model architecture and training our model.

In [6]:
# Now we're going to clean our dataset to prepare for our model.
# First, we will drop any rows with missing values in the columns we care about.
df = df.dropna(subset=["view_count", "likes", "dislikes", "comment_count"])

# We're also going to create some new features that might be useful for our model.
# Engagement features
df["like_ratio"] = df["likes"] / (df["view_count"] + 1)
df["comment_ratio"] = df["comment_count"] / (df["view_count"] + 1)

# Time features
df["published_hour"] = pd.to_datetime(df["publishedAt"]).dt.hour
df["trending_day"] = pd.to_datetime(df["trending_date"]).dt.dayofweek

In [7]:
# Now we're going to finalize a list of features we want to use for our model.
categorical_feature_names = [
    # we dont have ANY cateogorical features in this dataset, but for the sake of learning, i will include this list 
    # and show how to process them incase you have any categorical features in the future in your dataset anyway (because you will definitely have some in other real world applications)
]
numerical_feature_names = [
    "view_count",
    "likes",
    "dislikes",
    "comment_count",
    "like_ratio",
    "comment_ratio",
    "published_hour",
    "trending_day"
]
y = df["view_count"]  # Our target variable is view_count

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# usually, there are 2 types of columns that we're going to work with, categorical and numerical columns. In linear regression, and this goes for all sorts of models in general,
# the model can only deal with numbers, so all categories need to be converted into numbers using 1 hot encoding. This shouldn't be too difficult to do
tf_features = [] # features saved as actual processed tensorflow category objects/types
input_layers = {} # input layers saved as tensorflow objects/types. if we imagine our conceptual neural network models/diagrams, input layers are the nodes of the very first layer in the neural network, that do absolutely nothing except take in a value and pass it to the next layer.clear
# that function of the nodes, is EXACTLY what these input layers do, they just take in a value and pass it to the next layer

target = "view_count"
# as i said, we don't have any categorical features, but if we did, this is how we would process them (in the following lines of code)
for category in categorical_feature_names:
    inp = tf.keras.Input(shape=(1,), name=category) # we create an input layer for each category feature
    # this is basically a tensorflow object that holds a column of our data in a tf datatype/format (a tensor)
    # the reason we do this is because it makes it easier to process the data into encoding formats that tensorflow can understand, and allows us to actually feed the data so that models can be trained

    # for each category feature name, we need a vocabulary list (list of possible values this category can have), to create an encoded version of the category
    vocabulary = df[category].unique()
    # we then process each category into an actual TensorFlow categorical feature column type
    lookup = tf.keras.layers.StringLookup(vocabulary=vocabulary, output_mode='one_hot') # done so here!
    encoded = lookup(inp) # we encode the input layer using the lookup layer
    # now we just add the encoded layer as our final tf object to our list of tf-processed features
    tf_features.append(lookup)
    input_layers[category] = inp
    
for numerical_col in numerical_feature_names:
    inp = tf.keras.Input(shape=(1,), name = numerical_col) # this is the same as the previous (categorical) step, but a lot simpler since we don't need to do any encoding
    # so we create all these input layers for each numerical feature, and just directly add them to our list of tf-processed features since we arent encoding
    tf_features.append(inp)
    input_layers[numerical_col] = inp

2025-06-17 19:42:20.589413: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Model Architecture
- We're going to use TensorFlow (instead of scikit-learn) to gain deeper control over the model-building process. Scikit-learn is very good for quick prototyping, but TensorFlow allows us to explicitly define layers, loss functions, and training loops — making it ideal for more explicity hands-on, get-your-hands-dirty learning (and even extensibility toward more complex deep learning models later!)

In [9]:

# Remember, since this is just a linear regression model, we will just have 2 layers. The input and output layer, no hidden layers. 
# This is because linear regression is straight up just input -> output, it can be determined by a simple linear equation, its not a complex function like neural networks where you need hidden layers to learn complex internal patterns
# Now we will concatenate (flatten) all the input layers/tf_features into a single flat input layer (tensor) that will be fed into the model
x = tf.keras.layers.concatenate(tf_features) # this is the input layer of our model
output = tf.keras.layers.Dense(1)(x) # this is the output layer of our model, we just use a dense layer with 1 output since we only have 1 target variable (view_count)
# the (x) just means that we are feeding the concatenated input layer into the dense layer (and it learns the weight for each input feature) to get the output layer/predict the output (the linear regression)
model = tf.keras.Model(inputs=input_layers, outputs=output)
# remember the input_layers dictionary we made? it contains all the input layers we created for each feature, and contains all the nodes of the first layer of our model, meaning it makes up the input layer of our model, 
# so we just pass it in as our inputs to the model

# Now we will compile the model
model.compile(optimizer="adam", loss="mse")
# the optimizer function we used was adam, and the loss function we are using is mse.

Now our model has been compiled, we need to prepare our data for training (grouping, repeats, batching, etc etc).

In [10]:
def prepare_input(dataframe : pd.DataFrame): # this is an input function that will help prepare our data into a format that can be passed into the model for training
    # we will create a dictionary of inputs, where the keys are the feature names and the values are the corresponding columns from the dataframe
    # this format of the data makes it super easy to train models as thats what the model expects
    return {
        col: dataframe[col].tolist() for col in dataframe.columns if col != target
    }
# just for testing, lets get a preview of what col looks like, but its very large so we cant just dry print the whole thing:
input_data = prepare_input(df)
# lets just print the first 5 keys and values of the input_data dictionary
for key, value in list(input_data.items())[:5]:
    print(f"{key}: {value[:5]}")  # Print first 5 values for each key

video_id: ['J78aPJ3VyNs', '9nidKH8cM38', 'M9Pmf9AB4Mo', 'kgUV1MaD_M8', '49Z6Mv4_WCA']
title: ['I left youtube for a month and THIS is what happened.', "TAXI CAB SLAYER KILLS 'TO KNOW HOW IT FEELS'", 'Apex Legends | Stories from the Outlands – “The Endorsement”', 'Nines - Clout (Official Video)', "i don't know what im doing anymore"]
publishedAt: ['2020-08-11T16:34:06Z', '2020-08-11T20:00:45Z', '2020-08-11T17:00:10Z', '2020-08-10T18:30:28Z', '2020-08-11T20:24:34Z']
channelId: ['UCYzPXprvl5Y-Sf0g4vX-m6g', 'UCFMbX7frWZfuWdjAML0babA', 'UC0ZV6M2THA81QT9hrVWJG3A', 'UCvDkzrj8ZPlBqRd6fIxdhTw', 'UCtinbF-Q-fVthA0qrFQTgXQ']
channelTitle: ['jacksepticeye', 'Eleanor Neale', 'Apex Legends', 'Nines', 'CaseyNeistat']


In [ ]:
import time
start = time.time() # just going to keep track of how long it takes to train our model

# now we can train the model
X_df = df.drop(target, axis=1)
y = df[target].tolist() # get the target variable's values as a numpy array, which will also be used later to train the model

X_df_train, X_df_test, y_train, y_test = train_test_split( # here we're using scikit learn to split our dataset into training and testing sets. this will help a lot with evaulation later.
    X_df, y, test_size=0.2, random_state=42
)

X_train = prepare_input(X_df_train) # prepare the input data and store it (theres nothing wrong with continuing to use the same input_data variable, but I just wanted to show some of the common conventions used when working with linear regression models)
X_test = prepare_input(X_df_test) # same goes for the testing input, it also needs to be processed/prepared in this same way

# now to actually training the model
model.fit(
    X_train,
    y_train,
    epochs=5,               # Epochs are the number of times the model will go through the entire dataset, and we set it to 100 here, but you can change it based on your needs. 100 is just an arbitrary choice from me.
    batch_size=512,         # process 512 samples at a time instead of 32. as we know, the dataset is EXTREMELY LARGE, and having a smaller batch size of 32 (the default, and yes 32 is considered small in this context) will take FOREVER on a dataset of this kind of size. so we need to increase the amount of data that gets processed in a single batch.
    verbose=1               # show progress per epoch, verbosity is just the amount of detail/logs you want tensorflow to show you while training your model.
) 
# obviously we use the X_train and y_train data to train the model, and we will use the X_test and y_test data to evaluate the model later.

model.save("mymodel.keras") # saving our model just to be safe, we want instantaneous direct access to our trained model (and its weights and biases and everything) so that we can access it whenever and dont have to go through the pain of having to train it all over again from scratch from the beginning.

print(f"Training took {time.time() - start:.2f} seconds")

Now before we continue to evaluating the model, I'm actually going to show you some metrics first!

In [13]:
print(df.shape)  # printing the shape of the DataFrame to see how many rows and columns we have. this puts into perspective just how large this entire dataset of ours is.
print(model.summary())  # printing the model summary to see the architecture of our model, how many parameters it has, and other useful information about the model.

(268791, 20)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ view_count          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ likes (InputLayer)  │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dislikes            │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ comment_count       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ like_ratio          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ comment_ratio       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ published_hour      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ trending_day        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8)         │          0 │ view_count[0][0], │
│ (Concatenate)       │                   │            │ likes[0][0],      │
│                     │                   │            │ dislikes[0][0],   │
│                     │                   │            │ comment_count[0]… │
│                     │                   │            │ like_ratio[0][0], │
│                     │                   │            │ comment_ratio[0]… │
│                     │                   │            │ published_hour[0… │
│                     │                   │            │ trending_day[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1)         │          9 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9 (36.00 B)

 Trainable params: 9 (36.00 B)

 Non-trainable params: 0 (0.00 B)

None


As you can see, our dataset is massive. It has _____ amount of dataset rows/records/entries, with each having ___ fields of information.
That's exactly why it took such a long time (!) to train our model. (This value will likely not show for you, however, if you try training this model on your own by running this exact code too, it'll likely take you a similar amount of time).

Now, for model evaluation, it functions a little differently in linear regression models compared to other models.
For example, in classification, we can just measure the pure accuracy of how often the model guesses the correct class/category.
However, exact guesses like this are super unrealistic when it comes to linear regression (or any sort of regression at all for that matter!).
The aim of regression models is NEVER about being able to guess/estimate the EXACT numerical output/y. It's about minimizing the overall error (difference in estimated/guessed output and actual/real output).

So when we **evaulate** regression models, what we're really looking at and thinking about are metrics that measure and quantize error. 
Some of these error-measuring metrics are:
| Metric Name               | Description                                                                 | What It Tells You                                          | Example Value | Interpretation                                      |
|---------------------------|-----------------------------------------------------------------------------|-------------------------------------------------------------|----------------|------------------------------------------------------|
| MAE (Mean Absolute Error) | Average of absolute differences between predicted and actual views          | On average, how many views the model is off by             | 15,000 views   | Model is off by ~15K views per video on average      |
| MSE (Mean Squared Error)  | Average of squared differences between predicted and actual views           | Penalizes large errors more than small ones                | 625,000,000    | Large errors have a strong impact on model's score   |
| RMSE (Root Mean Squared Error) | Square root of MSE                                                   | Like MAE but more sensitive to large spikes in error       | 25,000 views   | Some predictions may be much farther off             |
| R² Score (R-squared)      | Proportion of view count variance explained by the model                    | 1.0 is perfect prediction, 0.0 means no useful learning     | 0.76           | Model explains 76% of the variation in video views   |

There are also other metrics, but these are the ones that best fit the scope of this project and are the most common, used and known metrics.

We will now evaluate our model.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # the scikit-learn library is infinitely useful. 
# it comes with a lot of handy tools, like the training and test data split i showed you earlier, as well as helper functions 
# for helping you calculate and determine ALL of the error-measuring metrics that i just previously mentioned and talked about

y_pred = model.predict(X_test).flatten() # gathering our predictions through our model using the test data that we held out during the training phase/stage of our model

mae = mean_absolute_error(y_test, y_pred) # calculating mae
mse = mean_squared_error(y_test, y_pred) # calculating mse
rmse = mse**0.5 # calculating rmse which is just the square root of mse
r2 = r2_score(y_test, y_pred) # calculating 

print(f"MAE:  {mae:,.0f} views")
# print(f"MSE: {mse:,.0f}") viewing mse by itself isn't that useful, since it doesnt really have a unit (or its unit is views^2 i suppose), and so it doesnt really illustrate or show anything useful on its own unless converted to rmse.
print(f"RMSE: {rmse:,.0f} views")
print(f"R² Score: {r2:.2f}")